In [2]:
import os, csv, pickle, json, pymongo, copy, re, time, datetime
import pandas as pd
from pymongo import MongoClient

In [3]:
pd.set_option('display.max_columns', 50)

In [4]:
jcr=pd.read_csv("../JCR/jcr2022.tsv",sep="\t")

In [5]:
len(jcr)

21430

In [6]:
jcr[-10:-1]

,rank,journalTitle,abbrJournal,year,edition,issn,totalCites,journalImpactFactor,impactFactorWithoutJournalSelfCites,fiveYearImpactFactor,immediacyIndex,citableItems,citedHalfLife,citingHalfLife,eigenFactorScore,articleInfluenceScore,citesCurrent,articlesCurrent,selfCites,impactFactorSelfCites,originalResearch,jifPercentile,normEigenFactor
21420,12425,AGENDA,AGENDA,2022,NaN,0002-0796,159,0.0,0.0,0.0,0.0,0,17.8,0.0,0.00005,0.114,NaN,NaN,NaN,NaN,NaN,0.0,0.01206
21421,12425,Abakos,ABAKOS,2022,NaN,2316-9451,7,0.0,0.0,0.0,0.0,18,0.0,10.7,0.00003,0.068,NaN,NaN,NaN,NaN,NaN,0.0,0.00733
21422,12425,Adalya,ADALYA,2022,NaN,1301-2746,75,0.0,0.0,0.0,0.0,19,0.0,16.2,0.00027,0.306,NaN,NaN,NaN,NaN,NaN,0.0,0.05846
21423,12425,Anduli,ANDULI,2022,NaN,1696-0270,35,0.0,0.0,0.0,0.0,15,0.0,14.6,0.00007,0.150,NaN,NaN,NaN,NaN,NaN,0.0,0.01555
21424,12425,Anemia,ANEMIA,2022,NaN,2090-1267,502,0.0,0.0,0.0,0.0,14,7.3,7.7,0.00030,0.550,NaN,NaN,NaN,NaN,NaN,0.0,0.06589
21425,12425,Appeal,APPEAL,2022,NaN,1205-612X,56,0.0,0.0,0.0,0.0,6,0.0,4.8,0.00019,0.550,NaN,NaN,NaN,NaN,NaN,0.0,0.04134
21426,12425,Atenas,ATENAS,2022,NaN,1682-2749,60,0.0,0.0,0.0,0.0,0,0.0,0.0,0.00007,0.034,NaN,NaN,NaN,NaN,NaN,0.0,0.01561
21427,12425,Atenea,ATENEA-CONCEPC,2022,NaN,0718-0462,105,0.0,0.0,0.0,0.0,35,8.8,15.8,0.00034,0.281,NaN,NaN,NaN,NaN,NaN,0.0,0.07463
21428,12425,AusArt,AUSART,2022,NaN,2340-9134,13,0.0,0.0,0.0,0.0,32,0.0,11.8,0.00002,0.017,NaN,NaN,NaN,NaN,NaN,0.0,0.00594


In [7]:
jt2if_all={}
for i in range(len(jcr)):
    if jt2if_all.get(jcr.iloc[i,1].lower()):
        if jt2if_all.get(jcr.iloc[i,1].lower())==0.0:
            jt2if_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
    else:
        jt2if_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
jt2if_all['aging']=jt2if_all.get('aging-us')

In [8]:
jt2if_brief={}
for i in range(len(jcr)):
    if jt2if_brief.get(jcr.iloc[i,2].lower()):
        if jt2if_brief.get(jcr.iloc[i,2].lower())==0.0:
            jt2if_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
    else:
        jt2if_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
jt2if_brief['aging']=jt2if_brief.get('aging-us')

In [9]:
list(jt2if_brief.items())[:2]

[('ca-cancer j clin', 286.13), ('lancet', 202.731)]

In [10]:
len(jt2if_brief)

21429

In [11]:
jt2if5_all={}
for i in range(len(jcr)):
    if jt2if5_all.get(jcr.iloc[i,1].lower()):
        if jt2if5_all.get(jcr.iloc[i,1].lower())==0.0:
            jt2if5_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
    else:
        jt2if5_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,9]
jt2if5_all['aging']=jt2if5_all.get('aging-us')

In [12]:
jt2if5_brief={}
for i in range(len(jcr)):
    if jt2if5_brief.get(jcr.iloc[i,2].lower()):
        if jt2if5_brief.get(jcr.iloc[i,2].lower())==0.0:
            jt2if5_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
    else:
        jt2if5_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,9]
jt2if5_brief['aging']=jt2if5_brief.get('aging-us')

In [13]:
list(jt2if5_brief.items())[:2]

[('ca-cancer j clin', 334.259), ('lancet', 130.838)]

In [16]:
client = MongoClient(username="hald",password="mclab236")

In [17]:
db = client.hald

In [18]:
collections_pubtator_info = db.pubtator_info

In [19]:
pubtator_info=list(collections_pubtator_info.find())

In [20]:
len(pubtator_info)

175189

In [21]:
pubtator_info[10]

{'_id': ObjectId('6393478a5303aaecac2745e7'),
 'PMID': '35794516',
 'OWN': 'NLM',
 'STAT': 'MEDLINE',
 'DCOM': '20220708',
 'LR': '20220708',
 'IS': '1471-227X (Electronic) 1471-227X (Linking)',
 'VI': '22',
 'IP': '1',
 'DP': '2022 Jul 6',
 'TI': 'Long-term prognosis and clinical course of choking-induced cardiac arrest in patients without the return of spontaneous circulation at hospital arrival: a population-based community study from the Shizuoka Kokuho Database.',
 'PG': '120',
 'LID': '10.1186/s12873-022-00676-8 [doi]',
 'AB': 'BACKGROUND: The risk of choking increases with aging, and the number of cases of choking-induced cardiac arrest is increasing. However, few studies have examined the prognosis of choking-induced cardiac arrest. The aim of this study was to reveal the rates of survival and dependence on devices in the long term after choking-induced cardiac arrest. METHODS: We analyzed data from the Shizuoka Kokuho Database, which consists of claims data of approximately 2.

In [22]:
pubtator_info_dict={}
for i in pubtator_info:
    pubtator_info_dict[i.get('PMID')]=i

In [23]:
collections_Relations_final=db.Relations_final

In [24]:
triplelist=list(collections_Relations_final.find())

In [25]:
triplelist[0]

{'_id': ObjectId('64611763a2e9cde11765b7be'),
 'PMID': '30781849',
 'entity1': 'Pulmonary Disease, Chronic Obstructive',
 'entity2': 'Inflammation',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tob

In [26]:
len(triplelist)

122404

In [27]:
be=['am','is','are','was','were','been','have been','has been']
bedict={}
for i in be:
    bedict[i]='be'

In [28]:
triplelist_takeoff=[]
transformed_triples=set()
box=[]
box2=[]
acount=0
for i in triplelist:
    maxlength=[]
    allennlp_list=[]
    if i.get('openie_final'):
        maxlength=i.get('openie_final')[0]
        for j in i.get('openie_final'):
            if len(j[1])>len(maxlength[1]):
                maxlength=j
        mid_list=maxlength[1].split(' ')
        for p,q in enumerate(mid_list):
            if bedict.get(q):
                mid_list[p]='be'
        maxlength[1]=' '.join(mid_list)
        for k in maxlength[1]:
            if k in maxlength[0].split(' ') or k in maxlength[2].split(' '):
                break
        else:
            mid=tuple([maxlength[0],maxlength[1].lower(),maxlength[2],i.get('PMID')])
            if mid not in transformed_triples:
                copydict={'triple':maxlength,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                          'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                          'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                          'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                            'transformed_triple':mid}
                triplelist_takeoff.append(copydict)
                transformed_triples.add(mid)
            if maxlength[1]=='associated' or maxlength[1]=='associate' :
                mid=tuple([maxlength[2],maxlength[1].lower(),maxlength[0],i.get('PMID')])
                if mid not in transformed_triples:
                    copydict={'triple':maxlength,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                              'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                              'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                              'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                'transformed_triple':mid}
                    triplelist_takeoff.append(copydict)
                    transformed_triples.add(mid)
        for j in i.get('openie_final'):
            if j==maxlength:
                continue
            judge=True
            for k in j[1].split(' '):
                if k not in maxlength[1].split(' '):
                    judge=False
                    break
            if not judge:
                mid_list=j[1].split(' ')
                for p,q in enumerate(mid_list):
                    if bedict.get(q):
                        mid_list[p]='be'
                j[1]=' '.join(mid_list)
                mid=tuple([j[0],j[1].lower(),j[2],i.get('PMID')])
                for k in j[1]:
                    if k in j[0].split(" ") or k in j[1].split(" "):
                        break
                else:
                    if mid not in transformed_triples:
                        copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                                  'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                                  'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                                  'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                                    'transformed_triple':mid}
                        triplelist_takeoff.append(copydict)
                        transformed_triples.add(mid)
                    if j[1]=='associated' or j[1]=='associate' :
                        mid=tuple([j[2],j[1].lower(),j[0],i.get('PMID')])
                        if mid not in transformed_triples:
                            copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                                      'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                                      'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                                      'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                        'transformed_triple':mid}
                            triplelist_takeoff.append(copydict)
                            transformed_triples.add(mid)
    if i.get('allennlp_final'):
        for j in i.get('allennlp_final'):
            if len(maxlength)>0 and j[1] in maxlength[1].split(' '):
                pass
            else:
                mid=tuple([j[0],j[1].lower(),j[2],i.get('PMID')])
                if j[1] in j[0].split(' ') or j[1] in j[2].split(' '):
                    continue
                if mid not in transformed_triples:
                    allennlp_list.append((j[0],j[2]))
                    copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                              'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                              'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                              'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                                'transformed_triple':mid}
                    triplelist_takeoff.append(copydict)
                    transformed_triples.add(mid)
                else:
                    box2.append(i)
                if j[1]=='associated' or j[1]=='associate' :
                    mid=tuple([j[2],j[1].lower(),j[0],i.get('PMID')])
                    if mid not in transformed_triples:
                        allennlp_list.append([j[2],j[0]])
                        copydict={'triple':j,'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                                  'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                                  'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                                  'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                    'transformed_triple':mid}
                        triplelist_takeoff.append(copydict)
                        transformed_triples.add(mid)
                    else:
                        box2.append(i)
    if i.get('networkx'):
        if len(maxlength)>0 and i.get('networkx')[1] in maxlength[1].split(' '):
            box.append(i)
            pass
        else:
            if i.get('networkx')[1] in i.get('networkx')[0].split(' ') or i.get('networkx')[1] in i.get('networkx')[2].split(' '):
                acount+=1
                continue
            mid=tuple([i.get('networkx')[0],i.get('networkx')[1].lower(),i.get('networkx')[2],i.get('PMID')])
            if mid not in transformed_triples:
                copydict={'triple':i.get('networkx'),'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                          'AB':i.get('AB'),'entity_one':i.get('target1'),'entity_one_type':i.get('target1_type'),
                          'entity_two':i.get('target2'),'entity_two_type':i.get('target2_type'),
                          'entity_one_transformed':i.get('entity1'),'entity_two_transformed':i.get('entity2'),
                            'transformed_triple':mid}
                triplelist_takeoff.append(copydict)
                transformed_triples.add(mid)
            if i.get('networkx')[1]=='associated' or i.get('networkx')[1]=='associate' :
                mid=tuple([i.get('networkx')[2],i.get('networkx')[1].lower(),i.get('networkx')[0],i.get('PMID')])
                if mid not in transformed_triples:
                    copydict={'triple':i.get('networkx'),'PMID':i.get('PMID'),'sentence':i.get('sentence'),
                              'AB':i.get('AB'),'entity_one':i.get('target2'),'entity_one_type':i.get('target2_type'),
                              'entity_two':i.get('target1'),'entity_two_type':i.get('target1_type'),
                              'entity_one_transformed':i.get('entity2'),'entity_two_transformed':i.get('entity1'),
                                'transformed_triple':mid}
                    triplelist_takeoff.append(copydict)
                    transformed_triples.add(mid)

In [29]:
i

{'_id': ObjectId('64611764a2e9cde1176795e1'),
 'PMID': '36923729',
 'entity1': 'Receptor-CD3 Complex, Antigen, T-Cell',
 'entity2': 'Infections',
 'sentence': 'These results suggest that although young and older donors retain classic TCR features for each epitope their repertoires are gradually changing with age, maintaining TCRs that are cross-reactive between these two common human viruses, one with recurrent infections and the other a persistent virus which frequently reactivates.',
 'AB': 'Older people have difficulty controlling infection with common viruses such as influenza A virus (IAV), RNA virus which causes recurrent infections due to a high rate of genetic mutation, and Epstein Barr virus (EBV), DNA virus which persists in B cells for life in the 95% of people that become acutely infected. We questioned whether changes in epitope-specific memory CD8 T cell receptor (TCR) repertoires to these two common viruses could occur with increasing age and contribute to waning immunit

In [30]:
len(box)

1295

In [31]:
len(box2)

2638

In [32]:
mid

('TCR', 'changing', 'infections', '36923729')

In [33]:
print(len(triplelist_takeoff))
print(len(transformed_triples))

141067
141067


In [34]:
triplelist_takeoff[0]

{'triple': ['Chronic obstructive pulmonary disease',
  'defined',
  'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obta

In [35]:
triplelist_takeoff_dict={}
for i in triplelist_takeoff:
    triplelist_takeoff_dict[(i.get('entity_one_transformed'),i.get('triple')[1],i.get('entity_two_transformed'),i.get('PMID'))]=triplelist_takeoff_dict.get((i.get('entity_one_transformed'),i.get('entity_two_transformed'),i.get('PMID')),[])+[i]

In [36]:
len(triplelist_takeoff_dict)

129833

In [37]:
triplelist_takeoffs=[]
a=[]
b=[]
c=[]
d=[]
for i,j in triplelist_takeoff_dict.items():
    if len(j)==1:
        triplelist_takeoffs.append(j[0])
    else:
        deprecated_set=set()
        for k in j:
            if k.get('entity_one').lower()==k.get('entity_one_transformed').lower() and k.get('entity_two').lower()==k.get('entity_two_transformed').lower():
                triplelist_takeoffs.append(k)
                a.append(k)
                break
        else:
            for k in j:
                if k.get('entity_one').lower()==k.get('entity_one_transformed').lower():
                    triplelist_takeoffs.append(k)
                    b.append(k)
                    break
            else:
                for k in j:
                    if k.get('entity_two').lower()==k.get('entity_two_transformed').lower():
                        c.append(k)
                        triplelist_takeoffs.append(k)
                        break
                else:
                    d.append(k)
                    triplelist_takeoffs.append(j[0])

In [38]:
names=locals()
for i in ['a','b','c','d']:
    print(len(names[i]))

0
0
0
0


In [39]:
len(triplelist_takeoffs)

129833

In [40]:
len(triplelist_takeoff)

141067

In [41]:
triplelist_takesoff=[]
for i in triplelist_takeoffs:
    if not re.search(r'\`|\~|\!|\@|\#|\$|\%|\^|\&|\*|\(|\)|\_|\+|\=|\{|\[|\}|\]|\;|\:|\'|\"|\,|\<|\.|\>|\/|\?',i.get('triple')[1]):
        if not re.search(r'(\s\d+\s)|(\b\d+\b)',i.get('triple')[1]): 
            triplelist_takesoff.append(i)

In [42]:
triplelistnewcopy=copy.deepcopy(triplelist_takesoff)

In [43]:
deprecated_relations=pd.read_csv('../classofrelations/deprecated.csv',header=None,names=['word'])

In [44]:
len(deprecated_relations)

209

In [45]:
deprecated_relations.head()

,word
0,% for
1,'s III be
2,)evaluate
3,#NAME?
4,#NAME?


In [46]:
deprecated_relations_dict={}
for i in deprecated_relations['word'].to_list():
    deprecated_relations_dict[i.lower()]=True

In [47]:
deprecated_relations_dict.get('have')

True

In [48]:
list(deprecated_relations_dict.items())[20]

('aforemention', True)

In [49]:
triplelistnewcopy[0]

{'triple': ['COPD', 'defined', 'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of

In [50]:
i

'promising'

In [51]:
bedict

{'am': 'be',
 'is': 'be',
 'are': 'be',
 'was': 'be',
 'were': 'be',
 'been': 'be',
 'have been': 'be',
 'has been': 'be'}

In [52]:
not_relations=pd.read_csv('../classofrelations/not_relations.csv',header=None,names=['word'])

In [53]:
len(not_relations)

61

In [54]:
not_relations.head()

,word
0,1p13 near
1,a of
2,abstractdiagnosing
3,acetylcholinesterase
4,apolipoprotein


In [55]:
not_relations_dict={}
for i in not_relations['word'].to_list():
    not_relations_dict[i.lower()]=True

In [56]:
list(not_relations_dict.items())[20]

('near 7q11 be', True)

In [58]:
triplelistnew=[]
for i in triplelistnewcopy: 
    if not deprecated_relations_dict.get(i.get('triple')[1].lower()) and not not_relations_dict.get(i.get('triple')[1].lower()):
        if i.get('triple')[1][0]=='\-':
            continue
        temp_triple=i.get('triple')[1].split(' ')
        for j,k in enumerate(temp_triple):
            if bedict.get(k.lower()):
                temp_triple[j]=bedict.get(k.lower())
        i['triple'][1]=' '.join(temp_triple)
        if i.get('triple')[1].lower()=='be' or bedict.get(i.get('triple')[1].lower()):
            continue
        if i.get('triple')[1].lower()=='means':
            i['triple'][1]='mean'
        if i.get('triple')[1].lower()=='meets':
            i['triple'][1]='meet'
        mid_judge=True
        for j in i.get('triple')[1].split(' '):
            if j.lower() not in i.get('triple')[0].lower().split(' ') and j.lower() not in i.get('triple')[2].lower().split(' '):
                pass
            else:
                mid_judge=False
        if mid_judge:
            triplelistnew.append(i)

In [59]:
len(triplelistnew)

120331

In [61]:
triplelistnew[10]

{'triple': ['Sarcopenia', 'associated', 'constipation'],
 'PMID': '34769606',
 'sentence': '(4) Conclusions: Sarcopenia and slow gait speed associated with constipation in community-dwelling older adults.',
 'AB': '(1) Background: As the clinical relevance of constipation and sarcopenia is not well studied, we aimed to investigate the association between them in older adults. (2) Methods: A cross-sectional study was conducted on 1278 community-dwelling older adults in South Korea. The Rome IV criteria were used to identify patients with clinically defined constipation, while sarcopenia was defined by the Asian Working Group for Sarcopenia consensus. The cohort was classified into three groups: no constipation, self-reported constipation only, and clinically defined constipation. (3) Results: The presence of constipation was associated with sarcopenia and slow gait speed (p < 0.001). After adjustment for possible covariates, the association with sarcopenia attenuated, while that for slo

In [62]:
triplelistnew[0]

{'triple': ['COPD', 'defined', 'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of

In [63]:
entity_set=set()
for i in triplelistnew:
    entity_set.add(i.get('entity_one_transformed'))
    entity_set.add(i.get('entity_two_transformed'))

In [64]:
len(entity_set)

6208

In [65]:
entitydict={}
for i,j in enumerate(triplelist):
    if j.get('entity1') in entity_set and j.get('entity2') in entity_set:
        if not entitydict.get(j.get('entity1')) and not entitydict.get(j.get('entity2')):
            entitydict[j.get('entity1')]=j.get('target1_type')
            entitydict[j.get('entity2')]=j.get('target2_type')
        else:
            if entitydict.get(j.get('entity1')):
                if re.search(j.get('target1_type'), entitydict.get(j.get('entity1'))) :
                    pass
                else:
                    entitydict[j.get('entity1')]+=";"+j.get('target1_type')
            else:
                entitydict[j.get('entity1')]=j.get('target1_type')
            if entitydict.get(j.get('entity2')):
                if re.search(j.get('target2_type'), entitydict.get(j.get('entity2'))) :
                    pass
                else:
                    entitydict[j.get('entity2')]+=";"+j.get('target2_type')
            else:
                entitydict[j.get('entity2')]=j.get('target2_type')

In [66]:
len(entitydict)

6208

In [67]:
entitydict_tolist={}
for i,j in enumerate(triplelist):
    if j.get('entity1') in entity_set and j.get('entity2') in entity_set:
        if not entitydict_tolist.get(j.get('entity1')) and not entitydict_tolist.get(j.get('entity2')):
            entitydict_tolist[j.get('entity1')]=[j.get('target1_type')]
            entitydict_tolist[j.get('entity2')]=[j.get('target2_type')]
        else:
            if entitydict_tolist.get(j.get('entity1')):
                if j.get('target1_type') in entitydict_tolist.get(j.get('entity1')):
                    pass
                else:
                    entitydict_tolist[j.get('entity1')]+=[j.get('target1_type')]
            else:
                entitydict_tolist[j.get('entity1')]=[j.get('target1_type')]
            if entitydict_tolist.get(j.get('entity2')):
                if j.get('target2_type') in  entitydict_tolist.get(j.get('entity2')) :
                    pass
                else:
                    entitydict_tolist[j.get('entity2')]+=[j.get('target2_type')]
            else:
                entitydict_tolist[j.get('entity2')]=[j.get('target2_type')]

In [68]:
entitydict.get('Lipopolysaccharides')

'Carbohydrate;Lipid;Toxin'

In [69]:
entitydict_tolist.get('Lipopolysaccharides')

['Carbohydrate', 'Lipid', 'Toxin']

In [70]:
len(entitydict)

6208

In [71]:
list(entitydict.items())[-30:-1]

[('FCRL3', 'Gene'),
 ('FCRLA', 'Gene'),
 ('TRIB2', 'Gene'),
 ('GSDMD', 'Gene'),
 ('BTN3A1', 'Gene'),
 ('CYTIP', 'Gene'),
 ('RASGRP1', 'Gene'),
 ('HIF1AN', 'Gene'),
 ('rs56105708', 'Mutation'),
 ('rs34833812', 'Mutation'),
 ('Trichothecenes', 'Toxin'),
 ('ABCD1', 'Gene'),
 ('PCYT2', 'Gene'),
 ('rs2549794', 'Mutation'),
 ('ERAP2', 'Gene'),
 ('SP3', 'Gene'),
 ('SP2', 'Gene'),
 ('PLEK', 'Gene'),
 ('FBXO7', 'Gene'),
 ('RPS20', 'Gene'),
 ('DR1', 'Gene'),
 ('NCAN', 'Gene'),
 ('BCAN', 'Gene'),
 ('Genital Neoplasms, Male', 'Disease'),
 ('Diastema', 'Disease'),
 ('Arginine Vasopressin', 'Peptide;Protein'),
 ('YBX1', 'Gene'),
 ('CSF1', 'Gene'),
 ('YPEL2', 'Gene')]

In [72]:
for i,j in entitydict.items():
    if re.search(';',j):
        print({i:j})

{'Ceramides': 'Carbohydrate;Lipid'}
{'Ristocetin': 'Carbohydrate;Peptide'}
{'Lipopolysaccharides': 'Carbohydrate;Lipid;Toxin'}
{'Sphingomyelins': 'Carbohydrate;Lipid'}
{'Daptomycin': 'Lipid;Peptide'}
{'Vancomycin': 'Carbohydrate;Peptide'}
{'Bleomycin': 'Carbohydrate;Peptide'}
{'Glycosylphosphatidylinositols': 'Carbohydrate;Lipid'}
{'Abciximab': 'Peptide;Protein'}
{'Angiotensin II': 'Peptide;Protein'}
{'Angiotensins': 'Peptide;Protein'}
{'Psychosine': 'Carbohydrate;Lipid'}
{'Glycolipids': 'Carbohydrate;Lipid'}
{'Gangliosides': 'Carbohydrate;Lipid'}
{'G(M1) Ganglioside': 'Carbohydrate;Lipid'}
{'G(M3) Ganglioside': 'Carbohydrate;Lipid'}
{'Glycosphingolipids': 'Carbohydrate;Lipid'}
{'Phospholipid Ethers': 'Carbohydrate;Lipid'}
{'Glycopeptides': 'Carbohydrate;Peptide'}
{'Cilastatin, Imipenem Drug Combination': 'Lipid;Pharmaceutical Preparations'}
{'Phalloidine': 'Peptide;Toxin'}
{'Globosides': 'Carbohydrate;Lipid'}
{'Glucosylceramides': 'Carbohydrate;Lipid'}
{'Cerebrosides': 'Carbohydrate;L

In [73]:
entitynum={}
num=1
for i in entitydict.keys():
    entitynum[i]=num
    num+=1

In [74]:
entitynum.get('APOE')

243

In [75]:
for i in triplelistnew:
    i['verb']=i.get('triple')[1]

In [76]:
entity_countnum={}
relation_countnum={}
tripleWeight={}
tripleWeightunique=set()
for i in triplelistnew:
    if i.get('entity_one_transformed')!=i.get('entity_two_transformed'):
        if (i.get('entity_one_transformed'),i.get('triple')[1].lower(),i.get('entity_two_transformed'),i.get('PMID')) not in tripleWeightunique:
            tripleWeightunique.add((i.get('entity_one_transformed'),i.get('triple')[1].lower(),i.get('entity_two_transformed'),i.get('PMID')))
            entity_countnum[i.get('entity_one_transformed')]=entity_countnum.get(i.get('entity_one_transformed'),0)+1
            entity_countnum[i.get('entity_two_transformed')]=entity_countnum.get(i.get('entity_two_transformed'),0)+1
            relation_countnum[i.get('triple')[1]]=relation_countnum.get(i.get('triple')[1],0)+1
            tripleWeight[(i.get('entity_one_transformed'),i.get('triple')[1].lower(), i.get('entity_two_transformed'))]=tripleWeight.get((i.get('entity_one_transformed'),i.get('triple')[1].lower(), i.get('entity_two_transformed')),0)+1

In [77]:
len(entity_countnum)

6208

In [78]:
len(tripleWeight)

96450

In [79]:
len(triplelistnew)

120331

In [80]:
triplelistnew[0]

{'triple': ['COPD', 'defined', 'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of

In [81]:
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
def cal_date(i):
    j=i
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    return date

In [82]:
for i in triplelistnew:
    if jt2if_brief.get(pubtator_info_dict.get(i.get('PMID')).get('TA').lower()):
        i['IF']=jt2if_brief.get(pubtator_info_dict.get(i.get('PMID')).get('TA').lower())
    elif jt2if_all.get(pubtator_info_dict.get(i.get('PMID')).get('JT').lower()):
        i['IF']=jt2if_all.get(pubtator_info_dict.get(i.get('PMID')).get('JT').lower())
    else:
        i['IF']=0.0
    if jt2if5_brief.get(pubtator_info_dict.get(i.get('PMID')).get('TA').lower()):
        i['IF5']=jt2if5_brief.get(pubtator_info_dict.get(i.get('PMID')).get('TA').lower())
    elif jt2if5_all.get(pubtator_info_dict.get(i.get('PMID')).get('JT').lower()):
        i['IF5']=jt2if5_all.get(pubtator_info_dict.get(i.get('PMID')).get('JT').lower())
    else:
        i['IF5']=0.0

In [83]:
triplelistnew[0]

{'triple': ['COPD', 'defined', 'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of

In [84]:
relation_interpolation_list=[]
relation_interpolation_dict={}
for i in triplelistnew:
    if (i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed')) not in relation_interpolation_dict.keys():
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]={'from':entitynum.get(i.get('entity_one_transformed'))-1,
                                   'to':entitynum.get(i.get('entity_two_transformed'))-1,'relation':i.get('triple')[1],
                                   'PMID':[i.get('PMID')],'sentence':[i.get('sentence')],
                                   'DP':[pubtator_info_dict.get(i.get('PMID')).get('DP')],
                                   'TI':[pubtator_info_dict.get(i.get('PMID')).get('TI')],
                                    'TA':[pubtator_info_dict.get(i.get('PMID')).get('TA')],
                                    'IF':[i.get('IF')],   'IF5':[i.get('IF5')],   
                                    'date':[cal_date(pubtator_info_dict.get(i.get('PMID')).get('DP'))],
                                    'first_entity_transformed':i.get('entity_one_transformed'),'second_entity_transformed':i.get('entity_two_transformed'),                                                     
                                    'first_entity':[i.get('entity_one')],'second_entity':[i.get('entity_two')],
                                    'first_type':entitydict_tolist.get(i.get('entity_one_transformed')),'second_type':entitydict_tolist.get(i.get('entity_two_transformed'))}
    else:
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['PMID']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('PMID')+[i.get('PMID')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['sentence']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('sentence')+[i.get('sentence')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['first_entity']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('first_entity')+[i.get('entity_one')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['second_entity']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('second_entity')+[i.get('entity_two')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['DP']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('DP')+[pubtator_info_dict.get(i.get('PMID')).get('DP')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['TI']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('TI')+[pubtator_info_dict.get(i.get('PMID')).get('TI')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['TA']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('TA')+[pubtator_info_dict.get(i.get('PMID')).get('TA')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['date']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('date')+[cal_date(pubtator_info_dict.get(i.get('PMID')).get('DP'))]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['IF']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('IF')+[i.get('IF')]
        relation_interpolation_dict[(i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))]['IF5']=relation_interpolation_dict.get((i.get('entity_one_transformed'),i.get('triple')[1], i.get('entity_two_transformed'))).get('IF5')+[i.get('IF5')]

In [85]:
list(relation_interpolation_dict.items())[10]

(('Sarcopenia', 'associated', 'Constipation'),
 {'from': 3,
  'to': 13,
  'relation': 'associated',
  'PMID': ['34769606'],
  'sentence': ['(4) Conclusions: Sarcopenia and slow gait speed associated with constipation in community-dwelling older adults.'],
  'DP': ['2021 Oct 21'],
  'TI': ['Status of Constipation and Its Association with Sarcopenia in Older Adults: A Population-Based Cohort Study.'],
  'TA': ['Int J Environ Res Public Health'],
  'IF': [4.614],
  'IF5': [4.798],
  'date': [20211021],
  'first_entity_transformed': 'Sarcopenia',
  'second_entity_transformed': 'Constipation',
  'first_entity': ['Sarcopenia'],
  'second_entity': ['constipation'],
  'first_type': ['Disease'],
  'second_type': ['Disease']})

In [200]:
len(relation_interpolation_dict)

96453

In [86]:
relation_interpolation_list=list(relation_interpolation_dict.values())

In [87]:
len(relation_interpolation_list)

96453

In [88]:
entity2content={}
for i,j in enumerate(triplelistnew):
    if j.get('entity_one_transformed') in entity2content.keys():
        entity2content[j.get('entity_one_transformed')]['sentence']=entity2content.get(j.get('entity_one_transformed')).get('sentence')+[j.get('sentence')]
        entity2content[j.get('entity_one_transformed')]['PMID']=entity2content.get(j.get('entity_one_transformed')).get('PMID')+[j.get('PMID')]
        entity2content[j.get('entity_one_transformed')]['entity']=entity2content.get(j.get('entity_one_transformed')).get('entity')+[j.get('entity_one')]
        entity2content[j.get('entity_one_transformed')]['DP']=entity2content.get(j.get('entity_one_transformed')).get('DP')+[pubtator_info_dict.get(j.get('PMID')).get('DP')]
        entity2content[j.get('entity_one_transformed')]['TI']=entity2content.get(j.get('entity_one_transformed')).get('TI')+[pubtator_info_dict.get(j.get('PMID')).get('TI')]
        entity2content[j.get('entity_one_transformed')]['TA']=entity2content.get(j.get('entity_one_transformed')).get('TA')+[pubtator_info_dict.get(j.get('PMID')).get('TA')]
        entity2content[j.get('entity_one_transformed')]['date']=entity2content.get(j.get('entity_one_transformed')).get('date')+[cal_date(pubtator_info_dict.get(j.get('PMID')).get('DP'))]
        entity2content[j.get('entity_one_transformed')]['IF']=entity2content.get(j.get('entity_one_transformed')).get('IF')+[j.get('IF')]
        entity2content[j.get('entity_one_transformed')]['IF5']=entity2content.get(j.get('entity_one_transformed')).get('IF5')+[j.get('IF5')]
    else:
        entity2content[j.get('entity_one_transformed')]={'entity_transformed':j.get('entity_one_transformed'),'entity':[j.get('entity_one')],'sentence':[j.get('sentence')],'PMID':[j.get('PMID')],
                                                         'DP':[pubtator_info_dict.get(j.get('PMID')).get('DP')],'TI':[pubtator_info_dict.get(j.get('PMID')).get('TI')],
                                                        'TA':[pubtator_info_dict.get(j.get('PMID')).get('TA')],'date':[cal_date(pubtator_info_dict.get(j.get('PMID')).get('DP'))],
                                                        'IF':[j.get('IF')],'IF5':[j.get('IF5')]}
    if j.get('entity_two_transformed') in entity2content.keys():
        entity2content[j.get('entity_two_transformed')]['sentence']=entity2content.get(j.get('entity_two_transformed')).get('sentence')+[j.get('sentence')]
        entity2content[j.get('entity_two_transformed')]['PMID']=entity2content.get(j.get('entity_two_transformed')).get('PMID')+[j.get('PMID')]
        entity2content[j.get('entity_two_transformed')]['entity']=entity2content.get(j.get('entity_two_transformed')).get('entity')+[j.get('entity_two')]
        entity2content[j.get('entity_two_transformed')]['DP']=entity2content.get(j.get('entity_two_transformed')).get('DP')+[pubtator_info_dict.get(j.get('PMID')).get('DP')]
        entity2content[j.get('entity_two_transformed')]['TI']=entity2content.get(j.get('entity_two_transformed')).get('TI')+[pubtator_info_dict.get(j.get('PMID')).get('TI')]
        entity2content[j.get('entity_two_transformed')]['TA']=entity2content.get(j.get('entity_two_transformed')).get('TA')+[pubtator_info_dict.get(j.get('PMID')).get('TA')]
        entity2content[j.get('entity_two_transformed')]['date']=entity2content.get(j.get('entity_two_transformed')).get('date')+[cal_date(pubtator_info_dict.get(j.get('PMID')).get('DP'))]
        entity2content[j.get('entity_two_transformed')]['IF']=entity2content.get(j.get('entity_two_transformed')).get('IF')+[j.get('IF')]
        entity2content[j.get('entity_two_transformed')]['IF5']=entity2content.get(j.get('entity_two_transformed')).get('IF5')+[j.get('IF5')]
    else:
        entity2content[j.get('entity_two_transformed')]={'entity_transformed':j.get('entity_two_transformed'),'entity':[j.get('entity_two')],'sentence':[j.get('sentence')],'PMID':[j.get('PMID')],
                                                         'DP':[pubtator_info_dict.get(j.get('PMID')).get('DP')],'TI':[pubtator_info_dict.get(j.get('PMID')).get('TI')],
                                                        'TA':[pubtator_info_dict.get(j.get('PMID')).get('TA')],'date':[cal_date(pubtator_info_dict.get(j.get('PMID')).get('DP'))],
                                                        'IF':[j.get('IF')],'IF5':[j.get('IF5')]}

In [89]:
len(entity2content)

6208

In [90]:
entity2list=list(entity2content.values())

In [91]:
len(entity2list)

6208

In [92]:
for i in entity2list:
    i['_id']=i.get('entity_transformed')

In [93]:
entitydict.get(entity2list[0].get('entity_transformed'))

'Disease'

In [94]:
for i in entity2list:
    i['type']=entitydict.get(i.get('entity_transformed')).split(';')

In [95]:
len(entity2list)

6208

In [96]:
json.dump(entity2list,open('results/step4/entityinfo.json','w'))

In [97]:
for i in entity2list:
    if i.get('entity_transformed')=='Lipopolysaccharides':
        print(i.get('type'))

['Carbohydrate', 'Lipid', 'Toxin']


In [98]:
list(relation_interpolation_dict.items())[:3]

[(('Pulmonary Disease, Chronic Obstructive', 'defined', 'Inflammation'),
  {'from': 0,
   'to': 1,
   'relation': 'defined',
   'PMID': ['30781849'],
   'sentence': ['(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.'],
   'DP': ['2019 Feb 13'],
   'TI': ['Chronic Obstructive Pulmonary Disease as a Main Factor of Premature Aging.'],
   'TA': ['Int J Environ Res Public Health'],
   'IF': [4.614],
   'IF5': [4.798],
   'date': [20190213],
   'first_entity_transformed': 'Pulmonary Disease, Chronic Obstructive',
   'second_entity_transformed': 'Inflammation',
   'first_entity': ['COPD'],
   'second_entity': ['inflammatory disorder'],
   'first_type': ['Disease'],
   'second_type': ['Disease']}),
 (('Anorexia', 'associate', 'Sarcopenia'),
  {'from': 2,
   'to': 3,
   'relation': 'associate',
   'PMID': ['30641897'],
   'sentence': ["(1) Background: Appetite loss in older people, the 'An

In [99]:
relation_interpolation_list[10]

{'from': 3,
 'to': 13,
 'relation': 'associated',
 'PMID': ['34769606'],
 'sentence': ['(4) Conclusions: Sarcopenia and slow gait speed associated with constipation in community-dwelling older adults.'],
 'DP': ['2021 Oct 21'],
 'TI': ['Status of Constipation and Its Association with Sarcopenia in Older Adults: A Population-Based Cohort Study.'],
 'TA': ['Int J Environ Res Public Health'],
 'IF': [4.614],
 'IF5': [4.798],
 'date': [20211021],
 'first_entity_transformed': 'Sarcopenia',
 'second_entity_transformed': 'Constipation',
 'first_entity': ['Sarcopenia'],
 'second_entity': ['constipation'],
 'first_type': ['Disease'],
 'second_type': ['Disease']}

In [100]:
len(relation_interpolation_list)

96453

In [101]:
json.dump(relation_interpolation_list,open('results/step4/relationinfo.json','w'))

In [102]:
relation_interpolation_list[100]

{'from': 5,
 'to': 96,
 'relation': 'associated',
 'PMID': ['34772307'],
 'sentence': ['Drowsiness/lethargy (OR = 1.48, 95% CI: 1.22-1.82), agitation (OR = 1.66, 95% CI: 1.11-2.50), coma (OR = 23.95, 95% CI: 17.05-33.64), bradycardia (OR = 2.29, 95% CI: 1.22-4.32), rhabdomyolysis (OR = 8.84, 95% CI: 3.71-21.03), hypothermia (OR = 4.1, 95% CI: 1.77-9.51), and hyperthermia 2.10 (OR = 2.10, 95% CI: 1.04-4.22) were likely associated with major outcomes or death.'],
 'DP': ['2021 Dec'],
 'TI': ['Prognostic factors of acetaminophen exposure in the United States: An analysis of 39,000 patients.'],
 'TA': ['Hum Exp Toxicol'],
 'IF': [3.247],
 'IF5': [3.045],
 'date': [20211201],
 'first_entity_transformed': 'Death',
 'second_entity_transformed': 'Rhabdomyolysis',
 'first_entity': ['death'],
 'second_entity': ['rhabdomyolysis'],
 'first_type': ['Disease'],
 'second_type': ['Disease']}

In [103]:
len(relation_interpolation_list)

96453

In [104]:
len(triplelistnew)

120331

In [105]:
csvf_entity = open("results/step4/entity.csv", "w", newline='', encoding='utf-8')
w_entity = csv.writer(csvf_entity)

In [106]:
w_entity.writerow(("entity:ID", "name","type","pagerank",":LABEL"))

37

In [107]:
for i,j in entitydict.items():
    w_entity.writerow((entitynum.get(i), i,j,int(entity_countnum.get(i)),j))

In [108]:
csvf_entity.close()

In [109]:
entitycsv=pd.read_csv('results/step4/entity.csv')

In [110]:
entitycsv[:3]

,entity:ID,name,type,pagerank,:LABEL
0,1,"Pulmonary Disease, Chronic Obstructive",Disease,903,Disease
1,2,Inflammation,Disease,3619,Disease
2,3,Anorexia,Disease,124,Disease


In [111]:
len(entitycsv)

6208

In [112]:
csvf_entity = open("results/step4/roles.csv", "w", newline='', encoding='utf-8')
w_entity = csv.writer(csvf_entity)

In [113]:
w_entity.writerow((':START_ID', ':END_ID', "relation", "weight",":TYPE" ))

41

In [114]:
entitynotsame={}
for i in triplelistnew:
    if i.get('entity_one_transformed')!=i.get('entity_two_transformed'):
        if not entitynotsame.get((i.get('entity_one_transformed'),i.get('triple')[1].lower(), i.get('entity_two_transformed'))) :
            w_entity.writerow((  entitynum.get(i.get('entity_one_transformed')),entitynum.get(i.get('entity_two_transformed')), i.get('triple')[1].lower(),int(tripleWeight.get((i.get('entity_one_transformed'),i.get('triple')[1].lower(), i.get('entity_two_transformed')))), i.get('triple')[1]   ))
        else:
            continue
        entitynotsame[(i.get('entity_one_transformed'),i.get('triple')[1].lower(), i.get('entity_two_transformed'))]=True

In [115]:
entitynum.get('APOE')

243

In [116]:
csvf_entity.close()

In [117]:
for i in triplelistnew:
    if i.get('sentence')=='Incidence of epilepsy was not statistically elevated among males, those with dementia, or individuals reporting a history of head injury or treatment for depression.':
        print(i)

{'triple': ['dementia', 'report', 'head injury'], 'PMID': '16870396', 'sentence': 'Incidence of epilepsy was not statistically elevated among males, those with dementia, or individuals reporting a history of head injury or treatment for depression.', 'AB': 'OBJECTIVE: To determine age-specific incidence and cumulative incidence of epilepsy in a well-defined cohort of elderly people, and to examine how rates of epilepsy are modified by sex, race, stroke, dementia, head injury, and depression. METHODS: The authors examined data from a reconstructed cohort based on 1919 community-dwelling volunteers, followed as part of a large ongoing prospective aging study. RESULTS: Age-specific incidence was 10.6 (per 100,000 person-years) between ages 45 and 59, 25.8 between ages 60 and 74, and 101.1 between ages 75 and 89. Cumulative incidence was 0.15% from age 45 to age 60, 0.38% to age 70, 1.01% to age 80, and 1.47% to age 90. In addition, the difference in cumulative incidence among African-Amer

In [119]:
roles=pd.read_csv("results/step4/roles.csv")

In [120]:
roles.head()

,:START_ID,:END_ID,relation,weight,:TYPE
0,1,2,defined,1,defined
1,3,4,associate,1,associate
2,4,3,associate,1,associate
3,5,6,recognized,1,recognized
4,5,6,increase,1,increase


In [121]:
relation_list=roles.relation.to_list()

In [122]:
type(relation_list)

list

In [123]:
relation_list[:3]

['defined', 'associate', 'associate']

In [124]:
len(relation_list)

96450

In [125]:
len(set(relation_list))

2695

In [126]:
relation_list_set=pd.read_csv('../classofrelations/relation_list_set.csv',header=None,names=['word'])

In [127]:
relation_list_set_dict={}
for i in relation_list_set['word'].to_list():
    relation_list_set_dict[i.lower()]=True

In [128]:
positive_relations=pd.read_csv('../classofrelations/positive-relations.csv',header=None,names=['word'])

In [129]:
positive_relations_dict={}
for i in positive_relations['word'].to_list():
    positive_relations_dict[i.lower()]=True

In [130]:
negative_relations=pd.read_csv('../classofrelations/negative-relations.csv',header=None,names=['word'])

In [131]:
negative_relations_dict={}
for i in negative_relations['word'].to_list():
    negative_relations_dict[i.lower()]=True

In [132]:
negative_relations['word'].to_list()[-5:-1]

['postpon', 'be potential targets for', 'impede', 'postpon']

In [133]:
triplelistnew[0]

{'triple': ['COPD', 'defined', 'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of

In [134]:
dropout=[]
for i in triplelistnew:
    if i.get('entity_two_type')=='Disease':
        if i.get('entity_one_type')!='Disease':
            if i.get('entity_one_transformed')!= i.get('entity_two_transformed'):
                if positive_relations_dict.get(i.get('triple')[1].lower()):
                    i['partofspeech']='positive'
                elif negative_relations_dict.get(i.get('triple')[1].lower()):
                    i['partofspeech']='negative'
                else:
                    i['partofspeech']='none'
                    dropout.append(i)

In [135]:
len(dropout)

8093

In [136]:
relation_list=[]
disease_list_raw=[]
for i in triplelistnew:
    if i.get('entity_two_type')=='Disease':
        if i.get('entity_one_type')!='Disease':
            if i.get('entity_one_transformed')!= i.get('entity_two_transformed'):
                relation_list.append(i.get('triple')[1].lower())
                disease_list_raw.append(i)

In [137]:
len(relation_list)

16528

In [138]:
len(disease_list_raw)

16528

In [139]:
new_relation=set()
for i in disease_list_raw:
    if not relation_list_set_dict.get(i.get('triple')[1].lower()):
        new_relation.add(i.get('triple')[1].lower())

In [140]:
len(new_relation)

15

In [141]:
new_relation

{'blocked',
 'disengage',
 'drifting',
 'engage',
 'expanded',
 'hyperactivated',
 'incorporated',
 'mistaken',
 'processed',
 'reduce nerve conduction abnormalities in',
 'reformed',
 'relapsed',
 'server',
 'span',
 'supplemented'}

In [142]:
len(relation_list_set['word'].to_list())

1588

In [143]:
relation_list_set_new=relation_list_set['word'].to_list()
for i in new_relation:
    relation_list_set_new.append(i)
pd.DataFrame(relation_list_set_new,columns=['word']).to_csv('../classofrelations/relation_list_set.csv')

In [144]:
pd.DataFrame(relation_list_set_new,columns=['word']).to_csv('results/step4/relation_list_set.csv')

In [141]:
new_relation

{'blocked',
 'disengage',
 'drifting',
 'engage',
 'expanded',
 'hyperactivated',
 'incorporated',
 'mistaken',
 'processed',
 'reduce nerve conduction abnormalities in',
 'reformed',
 'relapsed',
 'server',
 'span',
 'supplemented'}

# DEAL WITH new_relation!!!!!!!!!!!!
## Classify the new_relation list into posit, negat, and middle lists.

In [159]:
posit=[]
negat=['disengage','reduce nerve conduction abnormalities in','server',]
middle=['blocked','drifting','engage','expanded','hyperactivated','incorporated','mistaken','processed',
       'reformed','relapsed','span','supplemented']

In [160]:
for i in disease_list_raw:
    if i.get('triple')[1]=='supplemented':
        print(i)
        break

{'triple': ['PUFA', 'supplemented', 'osteoporosis'], 'PMID': '12020975', 'sentence': 'OBJECTIVE: To review the effect of a diet supplemented with polyunsaturated fatty acids (PUFA) on prevention or treatment of osteoporosis.', 'AB': 'OBJECTIVE: To review the effect of a diet supplemented with polyunsaturated fatty acids (PUFA) on prevention or treatment of osteoporosis. METHODS: MEDLINE (1966-April 2001), Allied Complementary Medicine (1985-2001), Cochrane Library and Database of Systematic Reviews (1st Quarter 2001) was searched. Five reviews and no systematic reviews were found on this topic in the Cochrane Library. Eleven relevant in-vivo studies were identified on the effect of these compounds on bone. Eight were animal studies and three were randomised control trials (RCT) in human. RESULTS: There are two classes of PUFA designated as n-3 and n-6 with alpha-linolenic acid (ALA). These two different types of PUFA differently influence prostaglandin formation and hence modulate bone

In [161]:
positive_relations_new=positive_relations['word'].to_list()
for i in posit:
    positive_relations_new.append(i)
pd.DataFrame(positive_relations_new,columns=['word']).to_csv('../classofrelations/positive-relations.csv')

In [162]:
negative_relations_new=negative_relations['word'].to_list()
for i in negat:
    negative_relations_new.append(i)
pd.DataFrame(negative_relations_new,columns=['word']).to_csv('../classofrelations/negative-relations.csv')

In [163]:
pd.DataFrame(positive_relations_new,columns=['word']).to_csv('results/step4/positive-relations.csv')
pd.DataFrame(negative_relations_new,columns=['word']).to_csv('results/step4/negative-relations.csv')

In [164]:
print(len(positive_relations_new))
print(len(negative_relations_new))

315
239


In [165]:
negative_relations['word'].to_list()[-5:-1]

['postpon', 'be potential targets for', 'impede', 'postpon']

In [166]:
positive_relations_new_dict={}
for i in positive_relations_new:
    positive_relations_new_dict[i.lower()]=True

In [167]:
negative_relations_new_dict={}
for i in negative_relations_new:
    negative_relations_new_dict[i.lower()]=True

In [168]:
dropout=[]
disease_list=[]
for i in disease_list_raw:
    if positive_relations_new_dict.get(i.get('triple')[1].lower()):
        i['partofspeech']='positive'
        disease_list.append(i)
    elif negative_relations_new_dict.get(i.get('triple')[1].lower()):
        i['partofspeech']='negative'
        disease_list.append(i)
    else:
        i['partofspeech']='none'
        dropout.append(i)

In [169]:
len(disease_list)

8439

In [170]:
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
for i in disease_list:
    j=pubtator_info_dict.get(i.get('PMID')).get('DP')
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    i['date']=date
    i['TA']=pubtator_info_dict.get(i.get('PMID')).get('TA')
    i['JT']=pubtator_info_dict.get(i.get('PMID')).get('JT')
    i['year']=int(str(date)[:4])

In [171]:
disease_list[-1]

{'triple': ['capsaicin', 'associated', 'gastrointestinal irritation'],
 'PMID': '36986266',
 'sentence': 'This review examines the current literature on capsaicin and Capsimax, a capsaicin supplement associated with reduced gastrointestinal irritation compared to capsaicin.',
 'AB': 'Impaired cognition is the primary symptom of dementia, which can lead to functional disability and reduced quality of life among an increasingly ageing population. Ageing is associated with increased oxidative stress, chronic low-grade systemic inflammation, and endothelial dysfunction, which reduces cerebrovascular function leading to cognitive decline. Chronic low-grade systemic inflammatory conditions, such as obesity, exacerbate this decline beyond normal ageing and predispose individuals to neurodegenerative diseases, such as dementia. Capsaicin, the major pungent molecule of chilli, has recently demonstrated improvements in cognition in animal models via activation of the transient receptor potential

In [172]:
for i in disease_list:
    if i.get('triple')[1]=='impede':
        print(i)
        break

{'triple': ['NOTCH1', 'impede', 'carcinogenesis'], 'PMID': '36658434', 'sentence': 'NOTCH1 mutant clones occupy the majority of normal human esophagus by middle age but are comparatively rare in esophageal cancers, suggesting NOTCH1 mutations drive clonal expansion but impede carcinogenesis.', 'AB': 'NOTCH1 mutant clones occupy the majority of normal human esophagus by middle age but are comparatively rare in esophageal cancers, suggesting NOTCH1 mutations drive clonal expansion but impede carcinogenesis. Here we test this hypothesis. Sequencing NOTCH1 mutant clones in aging human esophagus reveals frequent biallelic mutations that block NOTCH1 signaling. In mouse esophagus, heterozygous Notch1 mutation confers a competitive advantage over wild-type cells, an effect enhanced by loss of the second allele. Widespread Notch1 loss alters transcription but has minimal effects on the epithelial structure and cell dynamics. In a carcinogenesis model, Notch1 mutations were less prevalent in tu

In [173]:
len(disease_list)

8439

In [174]:
entity_pairs=['Gene',
 'RNA',
 'Protein',
 'Carbohydrate',
 'Lipid',
 'Peptide',
 'Pharmaceutical_Preparations',
 'Toxin',
 'Mutation',
 'Disease']

In [175]:
names=locals()
for i in entity_pairs:
    names[i+'_typecount']=set()
for j in disease_list:
    for i in entitydict_tolist.get(j.get('entity_one_transformed')):
        i=i.replace(' ','_')
        names[i+'_typecount'].add(j.get('entity_one_transformed'))

In [176]:
entitydict_tolist.get('Lipopolysaccharides')

['Carbohydrate', 'Lipid', 'Toxin']

In [177]:
for i in entity_pairs:
    print(i+' '+str(len(list(names[i+'_typecount']))))

Gene 1254
RNA 40
Protein 14
Carbohydrate 78
Lipid 80
Peptide 23
Pharmaceutical_Preparations 2
Toxin 1
Mutation 260
Disease 0


In [178]:
disease_list_copy=copy.deepcopy(disease_list)

In [179]:
disease_list_copy[0]

{'triple': ['alanine aminotransferase', 'recognized', 'mortality'],
 'PMID': '34063029',
 'sentence': '(1) Background: A significantly reduced alanine aminotransferase (ALT) level is being recognized as a risk factor of increasing mortality in the elderly in relation to frailty.',
 'AB': '(1) Background: A significantly reduced alanine aminotransferase (ALT) level is being recognized as a risk factor of increasing mortality in the elderly in relation to frailty. In the elderly, both frailty and ischemic stroke are not only common, but are also associated with mortality. The aim of this research was to investigate whether a significantly reduced ALT level increases the all-cause mortality rate in the elderly with ischemic stroke. (2) Methods: Between February 2014 and April 2019, a retrospective study of 901 patients with ischemic stroke admitted to a university-affiliated hospital was conducted. Cox proportional hazard regression was used to determine whether a significantly reduced AL

In [180]:
for i in disease_list_copy:
    if len(entitydict_tolist.get(i.get('entity_one_transformed')))>1:
        i['entity_one_type']='; '.join(entitydict_tolist.get(i.get('entity_one_transformed')))

In [181]:
list(entitydict_tolist.items())[0]

('Pulmonary Disease, Chronic Obstructive', ['Disease'])

In [183]:
disease_list_copy[0]

{'triple': ['alanine aminotransferase', 'recognized', 'mortality'],
 'PMID': '34063029',
 'sentence': '(1) Background: A significantly reduced alanine aminotransferase (ALT) level is being recognized as a risk factor of increasing mortality in the elderly in relation to frailty.',
 'AB': '(1) Background: A significantly reduced alanine aminotransferase (ALT) level is being recognized as a risk factor of increasing mortality in the elderly in relation to frailty. In the elderly, both frailty and ischemic stroke are not only common, but are also associated with mortality. The aim of this research was to investigate whether a significantly reduced ALT level increases the all-cause mortality rate in the elderly with ischemic stroke. (2) Methods: Between February 2014 and April 2019, a retrospective study of 901 patients with ischemic stroke admitted to a university-affiliated hospital was conducted. Cox proportional hazard regression was used to determine whether a significantly reduced AL

In [184]:
len(disease_list_copy)

8439

In [185]:
positive_relations=[]
negative_relations=[]
for i in disease_list_copy:
    if i.get('partofspeech')=='positive':
        positive_relations.append(i)
    else:
        negative_relations.append(i)

In [186]:
len(positive_relations)

7222

In [187]:
json.dump(positive_relations,open('results/step4/aging_relations.json','w'))
json.dump(negative_relations,open('results/step4/longevity_relations.json','w'))

In [188]:
json.dump(disease_list_copy,open('results/step4/disease_relation.json','w'))

# Statistics

In [209]:
for j in entity_pairs:
    j=j.replace('_',' ')
    names[j+'_aging']=set()
for i in disease_list:
    if i.get('partofspeech')=='positive':
        for j in entitydict_tolist.get(i.get('entity_one_transformed')):
            j=j.replace('_',' ')
            names[j+'_aging'].add(i.get('entity_one_transformed'))
for j in entity_pairs:    
    j=j.replace('_',' ')
    print(j+'\t'+str(len(names[j+'_aging'])))

Gene	1149
RNA	37
Protein	14
Carbohydrate	65
Lipid	71
Peptide	20
Pharmaceutical Preparations	2
Toxin	1
Mutation	256
Disease	0


In [210]:
for j in entity_pairs:    
    j=j.replace('_',' ')
    print(str(len(names[j+'_aging']))+',', end='')

1149,37,14,65,71,20,2,1,256,0,

In [211]:
for j in entity_pairs:
    j=j.replace('_',' ')
    names[j+'_longevity']=set()
for i in disease_list:
    if i.get('partofspeech')=='negative':
        for j in entitydict_tolist.get(i.get('entity_one_transformed')):
            j=j.replace('_',' ')
            names[j+'_longevity'].add(i.get('entity_one_transformed'))
for j in entity_pairs:    
    j=j.replace('_',' ')
    print(j+'\t'+str(len(names[j+'_longevity'])))

Gene	333
RNA	10
Protein	6
Carbohydrate	50
Lipid	39
Peptide	10
Pharmaceutical Preparations	2
Toxin	1
Mutation	7
Disease	0


In [212]:
Genedict=json.load(open('../dict/Genedict.json'))

In [213]:
list(Genedict.items())[:10]

[('A1BG', ['A1BG']),
 ('A1CF', ['A1CF']),
 ('A2M', ['A2M']),
 ('A2ML1', ['A2ML1']),
 ('A3GALT2', ['A3GALT2']),
 ('A4GALT', ['A4GALT']),
 ('A4GNT', ['A4GNT']),
 ('AAAS', ['AAAS']),
 ('AACS', ['AACS']),
 ('AADAC', ['AADAC'])]

In [214]:
Genedict_upper={}
for i,j in Genedict.items():
    Genedict_upper[i.upper()]=j

In [215]:
num_of_aging=set()
num_of_longevity=set()
for i in disease_list:
    if i.get('partofspeech')=='positive':
        num_of_aging.add(i.get('entity_one_transformed'))
    elif i.get('partofspeech')=='negative':
        num_of_longevity.add(i.get('entity_one_transformed'))
print(len(num_of_aging))
print(len(num_of_longevity))

1603
452


In [216]:
disease_entity_list=[]
disease_entity_triple=set()
disease_entity_dict={}
for i in disease_list:
    if i.get('partofspeech')=='positive':
        if i.get('entity_one_type')=='Gene':
            disease_entity_triple.add((i.get('entity_one_transformed'),i.get('entity_two_transformed'),i.get('PMID')))
            disease_entity_list.append(i)
            disease_entity_dict[i.get('entity_one_transformed')]=disease_entity_dict.get(i.get('entity_one_transformed'),0)+1

In [217]:
len(disease_entity_dict)

1149

In [218]:
disease_entity_dict_sorted = sorted(disease_entity_dict.items(), key=lambda x: x[1], reverse=True)

In [219]:
len(disease_entity_dict_sorted)

1149

In [220]:
for i in disease_entity_dict_sorted[:20]:
    print(i)

('INS', 412)
('CRP', 234)
('KL', 171)
('APOE', 170)
('MAPT', 147)
('SIRT1', 98)
('IL6', 84)
('ADIPOQ', 76)
('LMNA', 63)
('ALB', 62)
('WRN', 62)
('CD4', 59)
('LEP', 59)
('TP53', 45)
('CST3', 41)
('SIRT6', 39)
('SHBG', 38)
('ACE', 36)
('RECQL4', 34)
('BDNF', 34)


In [222]:
relation_interpolation_list[0]

{'from': 0,
 'to': 1,
 'relation': 'defined',
 'PMID': ['30781849'],
 'sentence': ['(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.'],
 'DP': ['2019 Feb 13'],
 'TI': ['Chronic Obstructive Pulmonary Disease as a Main Factor of Premature Aging.'],
 'TA': ['Int J Environ Res Public Health'],
 'IF': [4.614],
 'IF5': [4.798],
 'date': [20190213],
 'first_entity_transformed': 'Pulmonary Disease, Chronic Obstructive',
 'second_entity_transformed': 'Inflammation',
 'first_entity': ['COPD'],
 'second_entity': ['inflammatory disorder'],
 'first_type': ['Disease'],
 'second_type': ['Disease']}

In [223]:
triplelistnew[0]

{'triple': ['COPD', 'defined', 'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of

In [224]:
direction=set()
direction_list=[]
direction_type=[]
for i in triplelistnew:
    for j in entitydict_tolist.get(i.get('entity_one_transformed')):
        for k in entitydict_tolist.get(i.get('entity_two_transformed')):
            if (i.get('entity_one_transformed'),(i.get('entity_two_transformed')),j,k) not in direction:
                direction.add((i.get('entity_one_transformed'),(i.get('entity_two_transformed')),j,k))
                direction_list.append((i.get('entity_one_transformed'),(i.get('entity_two_transformed'))))
                direction_type.append((j.replace(' ','_'),k.replace(' ','_')))

In [225]:
len(direction_list)

51323

In [226]:
type_dict={}
type_dict_ensure={}
for i in range(len(direction_list)):
    if not type_dict_ensure.get((direction_list[i][0],direction_list[i][1],direction_type[i][0],direction_type[i][1])):
        type_dict[(direction_list[i][0],direction_list[i][1])]=tuple((direction_type[i][0],direction_type[i][1]))
        type_dict_ensure[(direction_list[i][0],direction_list[i][1],direction_type[i][0],direction_type[i][1])]=tuple((direction_type[i][0],direction_type[i][1]))

In [227]:
len(type_dict)

50900

In [228]:
entity_pairs

['Gene',
 'RNA',
 'Protein',
 'Carbohydrate',
 'Lipid',
 'Peptide',
 'Pharmaceutical_Preparations',
 'Toxin',
 'Mutation',
 'Disease']

In [229]:
entity_pairs=['Gene',
 'RNA',
 'Protein',
 'Carbohydrate',
 'Lipid',
 'Peptide',
 'Pharmaceutical_Preparations',
 'Toxin',
 'Mutation',
 'Disease']

In [230]:
entity_pairs_dict={}
entity_pairs_dict_num={}
for i in entity_pairs:
    temp={}
    temp2={}
    for j in entity_pairs:
        temp.update({j:0})
        temp2.update({j:set()})
    entity_pairs_dict_num[i]=temp
    entity_pairs_dict[i]=temp2

In [231]:
entity_pairs_dict

{'Gene': {'Gene': set(),
  'RNA': set(),
  'Protein': set(),
  'Carbohydrate': set(),
  'Lipid': set(),
  'Peptide': set(),
  'Pharmaceutical_Preparations': set(),
  'Toxin': set(),
  'Mutation': set(),
  'Disease': set()},
 'RNA': {'Gene': set(),
  'RNA': set(),
  'Protein': set(),
  'Carbohydrate': set(),
  'Lipid': set(),
  'Peptide': set(),
  'Pharmaceutical_Preparations': set(),
  'Toxin': set(),
  'Mutation': set(),
  'Disease': set()},
 'Protein': {'Gene': set(),
  'RNA': set(),
  'Protein': set(),
  'Carbohydrate': set(),
  'Lipid': set(),
  'Peptide': set(),
  'Pharmaceutical_Preparations': set(),
  'Toxin': set(),
  'Mutation': set(),
  'Disease': set()},
 'Carbohydrate': {'Gene': set(),
  'RNA': set(),
  'Protein': set(),
  'Carbohydrate': set(),
  'Lipid': set(),
  'Peptide': set(),
  'Pharmaceutical_Preparations': set(),
  'Toxin': set(),
  'Mutation': set(),
  'Disease': set()},
 'Lipid': {'Gene': set(),
  'RNA': set(),
  'Protein': set(),
  'Carbohydrate': set(),
  'Lipi

In [232]:
for i in range(len(direction_list)):
        if direction_list[i][0] not in entity_pairs_dict.get(direction_type[i][1]).get(direction_type[i][0]):
            entity_pairs_dict[direction_type[i][1]][direction_type[i][0]].add(direction_list[i][0])
            entity_pairs_dict_num[direction_type[i][1]][direction_type[i][0]]+=1

In [233]:
entity_pairs_dict_num

{'Gene': {'Gene': 1054,
  'RNA': 33,
  'Protein': 9,
  'Carbohydrate': 50,
  'Lipid': 45,
  'Peptide': 19,
  'Pharmaceutical_Preparations': 0,
  'Toxin': 1,
  'Mutation': 214,
  'Disease': 694},
 'RNA': {'Gene': 37,
  'RNA': 6,
  'Protein': 0,
  'Carbohydrate': 1,
  'Lipid': 0,
  'Peptide': 0,
  'Pharmaceutical_Preparations': 0,
  'Toxin': 0,
  'Mutation': 0,
  'Disease': 33},
 'Protein': {'Gene': 31,
  'RNA': 0,
  'Protein': 2,
  'Carbohydrate': 5,
  'Lipid': 3,
  'Peptide': 0,
  'Pharmaceutical_Preparations': 0,
  'Toxin': 0,
  'Mutation': 0,
  'Disease': 74},
 'Carbohydrate': {'Gene': 150,
  'RNA': 3,
  'Protein': 3,
  'Carbohydrate': 63,
  'Lipid': 33,
  'Peptide': 7,
  'Pharmaceutical_Preparations': 2,
  'Toxin': 0,
  'Mutation': 6,
  'Disease': 283},
 'Lipid': {'Gene': 163,
  'RNA': 2,
  'Protein': 2,
  'Carbohydrate': 26,
  'Lipid': 66,
  'Peptide': 4,
  'Pharmaceutical_Preparations': 0,
  'Toxin': 1,
  'Mutation': 15,
  'Disease': 308},
 'Peptide': {'Gene': 47,
  'RNA': 0,
  'P

In [234]:
countnum=0
for i in entity_pairs_dict_num.values():
    for j in i.values():
        countnum+=j
print(countnum)

8592


In [235]:
relation_display=[]
for i,j in entity_pairs_dict_num.items():
    for k,l in j.items():
        relation_display.append([k,i+' ',l])

In [236]:
j

{'Gene': 1720,
 'RNA': 57,
 'Protein': 20,
 'Carbohydrate': 103,
 'Lipid': 105,
 'Peptide': 32,
 'Pharmaceutical_Preparations': 7,
 'Toxin': 3,
 'Mutation': 348,
 'Disease': 2152}

In [237]:
relation_display

[['Gene', 'Gene ', 1054],
 ['RNA', 'Gene ', 33],
 ['Protein', 'Gene ', 9],
 ['Carbohydrate', 'Gene ', 50],
 ['Lipid', 'Gene ', 45],
 ['Peptide', 'Gene ', 19],
 ['Pharmaceutical_Preparations', 'Gene ', 0],
 ['Toxin', 'Gene ', 1],
 ['Mutation', 'Gene ', 214],
 ['Disease', 'Gene ', 694],
 ['Gene', 'RNA ', 37],
 ['RNA', 'RNA ', 6],
 ['Protein', 'RNA ', 0],
 ['Carbohydrate', 'RNA ', 1],
 ['Lipid', 'RNA ', 0],
 ['Peptide', 'RNA ', 0],
 ['Pharmaceutical_Preparations', 'RNA ', 0],
 ['Toxin', 'RNA ', 0],
 ['Mutation', 'RNA ', 0],
 ['Disease', 'RNA ', 33],
 ['Gene', 'Protein ', 31],
 ['RNA', 'Protein ', 0],
 ['Protein', 'Protein ', 2],
 ['Carbohydrate', 'Protein ', 5],
 ['Lipid', 'Protein ', 3],
 ['Peptide', 'Protein ', 0],
 ['Pharmaceutical_Preparations', 'Protein ', 0],
 ['Toxin', 'Protein ', 0],
 ['Mutation', 'Protein ', 0],
 ['Disease', 'Protein ', 74],
 ['Gene', 'Carbohydrate ', 150],
 ['RNA', 'Carbohydrate ', 3],
 ['Protein', 'Carbohydrate ', 3],
 ['Carbohydrate', 'Carbohydrate ', 63],
 ['

In [238]:
entity_pairs

['Gene',
 'RNA',
 'Protein',
 'Carbohydrate',
 'Lipid',
 'Peptide',
 'Pharmaceutical_Preparations',
 'Toxin',
 'Mutation',
 'Disease']

In [239]:
for i in entity_pairs:
    xs=[]
    for j in relation_display:
        if j[0]==i.replace(' ','_'):
            xs.append(str(j[-1]))
    print('\t'.join(xs))

1054	37	31	150	163	47	0	20	130	1720
33	6	0	3	2	0	0	2	1	57
9	0	2	3	2	1	0	0	0	20
50	1	5	63	26	5	0	0	2	103
45	0	3	33	66	9	0	2	8	105
19	0	0	7	4	7	0	0	1	32
0	0	0	2	0	1	0	0	0	7
1	0	0	0	1	0	0	2	0	3
214	0	0	6	15	1	0	0	94	348
694	33	74	283	308	90	13	19	142	2152


In [240]:
all_es=json.load(open('results/step2/all_es_lack.json'))

In [252]:
entity2aging={}
entity2longevity={}
for i in triplelistnew:
    if i.get('entity_two_type')=='Disease':
        if i.get('entity_one_type')!='Disease':
            if i.get('entity_one_transformed')!= i.get('entity_two_transformed'):
                if positive_relations_new_dict.get(i.get('triple')[1].lower()):
                    entity2aging[i.get('entity_one_transformed')]=True
                elif negative_relations_new_dict.get(i.get('triple')[1].lower()):
                    entity2longevity[i.get('entity_one_transformed')]=True

In [253]:
len(entity2aging)

1603

In [254]:
len(entity2longevity)

452

In [255]:
aaaa=0
for i in entity2aging.keys():
    if entity2longevity.get(i):
        aaaa+=1
print(aaaa)

316


In [256]:
for i in all_es:
    if entitydict.get(i.get('entity')):
        i['network']=True
    else:
        i['network']=False
    if entity2aging.get(i.get('entity')):
        i['aging']=True
    else:
        i['aging']=False
    if entity2longevity.get(i.get('entity')):
        i['longevity']=True
    else:
        i['longevity']=False

In [257]:
json.dump(all_es,open('results/step4/all_es.json','w'))

In [258]:
len(all_es)

11127

In [260]:
entity_es_set=set()
for i in all_es:
    entity_es_set.add(i.get('entity'))

In [261]:
len(list(entity_es_set))

11080

In [265]:
names=locals()
for i in entity_pairs:
    names[i+'_all']=[]
    for j in all_es:
        if j.get('type')==i.replace('_',' '):
            names[i+'_all'].append(j)

In [266]:
len(Gene_all)

4988

In [267]:
len(Disease_all)

3352

In [268]:
Gene_all_sorted=sorted(Gene_all,key=lambda x:x['literature'],reverse=True)

In [270]:
for i in entity_pairs:
    print('\n')
    print(i)
    names[i+'_all_sorted']=sorted(names[i+'_all'],key=lambda x:x['literature'],reverse=True)
    names[i+'_all_sorted_num']=0
    for j in names[i+'_all_sorted'][:10]:
        print("'"+j.get('entity')+"',")
for i in entity_pairs:
    print('\n')
    print(i)
    for j in names[i+'_all_sorted'][:10]:
        print(str(j.get('literature'))+",")
        names[i+'_all_sorted_num']+=j.get('literature')
for i in entity_pairs:
    print('\n')
    print(i)
    cxv=0
    for j in names[i+'_all_sorted'][:10]:
        z=j.get('literature')*100/names[i+'_all_sorted_num']
        print(str(round(z,2))+",")



Gene
'INS',
'CRP',
'APOE',
'IL6',
'CD4',
'TNF',
'MAPT',
'ALB',
'SIRT1',
'CD8A',


RNA
'H19',
'MIR34A',
'TERC',
'MIR21',
'MIR146A',
'MIR155',
'MIR221',
'MIR132',
'MIR29A',
'HCCAT5',


Protein
'Infliximab',
'Rituximab',
'Bevacizumab',
'Denosumab',
'Trastuzumab',
'Palivizumab',
'Cetuximab',
'Nivolumab',
'Adalimumab',
'Ranibizumab',


Carbohydrate
'Glucose',
'Ascorbic Acid',
'Hyaluronic Acid',
'Sugars',
'Polysaccharides',
'Heparin',
'Digoxin',
'Lipopolysaccharides',
'Doxorubicin',
'Fluorodeoxyglucose F18',


Lipid
'Cholesterol',
'Triglycerides',
'Fatty Acids',
'Fatty Acids, Unsaturated',
'Fatty Acids, Omega-3',
'Lipofuscin',
'Lipopolysaccharides',
'Phospholipids',
'Cholecalciferol',
'Docosahexaenoic Acids',


Peptide
'Glutathione',
'Cyclosporine',
'Vancomycin',
'Thyrotropin',
'Ghrelin',
'Bleomycin',
'Enalapril',
'Glutathione Disulfide',
'Teriparatide',
'Dipeptides',


Pharmaceutical_Preparations
'Trimethoprim, Sulfamethoxazole Drug Combination',
'Amoxicillin-Potassium Clavulanate Combina

In [271]:
for i in entity_pairs:
    print('\n')
    print(i)
    print(names[i+'_all_sorted_num'])



Gene
11045


RNA
302


Protein
703


Carbohydrate
5898


Lipid
6694


Peptide
1304


Pharmaceutical_Preparations
168


Toxin
267


Mutation
221


Disease
122598


In [272]:
for i in entity_pairs:
    print(i)
    print(len(names[i+'_all']))


Gene
4988
RNA
350
Protein
31
Carbohydrate
204
Lipid
191
Peptide
79
Pharmaceutical_Preparations
11
Toxin
19
Mutation
1902
Disease
3352


In [273]:
for i in entity_pairs:
    print(i)
    axc=set()
    for j in names[i+'_all']:
        for k in j.get('PMID'):
            axc.add(k)
    print(len(axc))


Gene
26348
RNA
666
Protein
727
Carbohydrate
8065
Lipid
7421
Peptide
1597
Pharmaceutical_Preparations
149
Toxin
273
Mutation
1494
Disease
149887


In [274]:
all_es_result=json.load(open('results/step2/all_es_result.json'))

In [275]:
for i in entity_pairs:
    names[i.replace(' ','_')+'_sentences']=set()
for i in all_es_result:
    for j in i.get('sentence'):
        for k in j:
            if re.search('RNA',i.get('type')):
                names['RNA'.replace(' ','_')+'_sentences'].add(k)
            else:
                names[i.get('type').replace(' ','_')+'_sentences'].add(k)

In [276]:
for i in entity_pairs:
    print(i)
    print(len(names[i.replace(' ','_')+'_sentences']))

Gene
79242
RNA
2057
Protein
1738
Carbohydrate
15827
Lipid
15240
Peptide
3139
Pharmaceutical_Preparations
239
Toxin
462
Mutation
3476
Disease
635172


# Entity_Info.json

In [287]:
all_es_result_dict={}
for i in all_es_result:
    if not all_es_result_dict.get(i.get('entity')):
        all_es_result_dict[i.get('entity')]=i

In [288]:
all_es_result[10000]

{'entity': 'Mediastinal Neoplasms',
 'PMID': ['31393382'],
 'target': [['mediastinal tumors', 'mediastinal tumors']],
 'date': [20190801],
 'type': 'Disease',
 'sentence': [['Medical decisions should be well-planned to improve prognosis and reduce complications of mediastinal tumors.',
   "In this study, we analyzed the clinical presentations of pediatric mediastinal tumors and their correlation with long-term clinical outcome.Forty patients under 18 years of age diagnosed with mediastinal tumors at China Medical University Children's Hospital between 2001 and 2016 were enrolled."]],
 '_id': 'Disease_Mediastinal Neoplasms',
 'IF': [1.817],
 'IF5': [2.227],
 'TA': ['Medicine (Baltimore)'],
 'JT': ['Medicine'],
 'year': [2019]}

In [296]:
all_es_dict={}
for i in all_es:
    if not i.get('url'):
        i['url']=i.get('NCBI_URL')
    j={'entity':i.get('entity'),'type':i.get('type'),'PMID':i.get('PMID'),
       'sentence':all_es_result_dict.get(i.get('entity')).get('sentence'),
      'numbers of articles':i.get('literature'),
       'JT':all_es_result_dict.get(i.get('entity')).get('JT'),
       'TA':all_es_result_dict.get(i.get('entity')).get('TA'),
       'IF':all_es_result_dict.get(i.get('entity')).get('IF'),
       'IF5':all_es_result_dict.get(i.get('entity')).get('IF5'),
       'year':all_es_result_dict.get(i.get('entity')).get('year'),
       'date':all_es_result_dict.get(i.get('entity')).get('date'),
       'alias names':i.get('Also_known_as',''),
       'description':i.get('Summary',''),
       'url':i.get('url',''),
       'NCBI gene ID':i.get('NCBI_ID',''),
       'mutation position':i.get('position',''),
       'mutation alleles':i.get('Alleles',''),
       'MeSH ID':i.get('MeshID',''),
       'relation':i.get('network'),
       'aging biomarker':i.get('aging'),
       'longevity biomarker':i.get('longevity')}
    if not all_es_dict.get(i.get('entity')):
        all_es_dict[i.get('entity')]=[j]
    else:
        all_es_dict[i.get('entity')]=all_es_dict.get(i.get('entity'))+[j]

In [297]:
len(all_es_dict)

11080

In [308]:
all_es_length=set()
for i,j in all_es_dict.items():
    for k in j[0].get('PMID'):
        all_es_length.add(k)
print(len(all_es_length))

155774


In [313]:
json.dump(all_es_dict,open('results/step5/Entity_Info.json','w'))

# Relation_Info.json

In [301]:
triplelistnew[0]

{'triple': ['COPD', 'defined', 'inflammatory disorder'],
 'PMID': '30781849',
 'sentence': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem.',
 'AB': '(1) Background: Chronic obstructive pulmonary disease (COPD) is defined as an inflammatory disorder that presents an increasingly prevalent health problem. Accelerated aging has been examined as a pathologic mechanism of many chronic diseases like COPD. We examined whether COPD is combined with accelerated aging, studying two hormones, dehydroepiandrosterone (DHEA) and growth hormone (GH), known to be characteristic biological markers of aging. (2) Methods: Data were collected from 119 participants, 70 (58.8%) COPD patients and 49 (41.2%) from a health control group over the period of 2014-2016 in a spirometry program. Information about their medical history, tobacco use, and blood tests was obtained. (3) Results: The average age of

In [316]:
Relation_Info_list=[]
Relation_Info_dict={}
for i in triplelistnew:
    if i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed') not in Relation_Info_dict.keys():
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]={
            'source entity':i.get('entity_one_transformed'),
            'relationship':i.get('triple')[1],
            'target entity':i.get('entity_two_transformed'),
            'sentence':[i.get('sentence')],
            'source':[i.get('entity_one')],
            'target':[i.get('entity_two')],
            'source type':entitydict_tolist.get(i.get('entity_one_transformed')),
            'target type':entitydict_tolist.get(i.get('entity_two_transformed')),
            'PMID':[i.get('PMID')],
            'DP':[pubtator_info_dict.get(i.get('PMID')).get('DP')],
            'date':[cal_date(pubtator_info_dict.get(i.get('PMID')).get('DP'))],
            'TI':[pubtator_info_dict.get(i.get('PMID')).get('TI')],
            'TA':[pubtator_info_dict.get(i.get('PMID')).get('TA')],
            'IF':[i.get('IF')],
            'IF5':[i.get('IF5')]}
    else:
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['PMID']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('PMID')+[i.get('PMID')]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['sentence']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('sentence')+[i.get('sentence')]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['source']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('source')+[i.get('entity_one')]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['target']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('target')+[i.get('entity_two')]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['DP']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('DP')+[pubtator_info_dict.get(i.get('PMID')).get('DP')]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['TI']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('TI')+[pubtator_info_dict.get(i.get('PMID')).get('TI')]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['TA']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('TA')+[pubtator_info_dict.get(i.get('PMID')).get('TA')]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['date']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('date')+[cal_date(pubtator_info_dict.get(i.get('PMID')).get('DP'))]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['IF']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('IF')+[i.get('IF')]
        Relation_Info_dict[i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')]['IF5']=Relation_Info_dict.get(i.get('entity_one_transformed')+'-'+i.get('triple')[1]+'-'+i.get('entity_two_transformed')).get('IF5')+[i.get('IF5')]

In [325]:
list(Relation_Info_dict.items())[10]

('Sarcopenia-associated-Constipation',
 {'source entity': 'Sarcopenia',
  'relationship': 'associated',
  'target entity': 'Constipation',
  'sentence': ['(4) Conclusions: Sarcopenia and slow gait speed associated with constipation in community-dwelling older adults.'],
  'source': ['Sarcopenia'],
  'target': ['constipation'],
  'source type': ['Disease'],
  'target type': ['Disease'],
  'PMID': ['34769606'],
  'DP': ['2021 Oct 21'],
  'date': [20211021],
  'TI': ['Status of Constipation and Its Association with Sarcopenia in Older Adults: A Population-Based Cohort Study.'],
  'TA': ['Int J Environ Res Public Health'],
  'IF': [4.614],
  'IF5': [4.798]})

In [317]:
len(Relation_Info_dict)

96453

In [318]:
Relation_Info_length=set()
for i,j in Relation_Info_dict.items():
    for k in j.get('PMID'):
        Relation_Info_length.add(k)
print(len(Relation_Info_length))

41708


In [319]:
json.dump(Relation_Info_dict,open('results/step5/Relation_Info.json','w'))

# Aging_Biomarkers.json

In [320]:
len(positive_relations)

7222

In [321]:
positive_relations[0]

{'triple': ['alanine aminotransferase', 'recognized', 'mortality'],
 'PMID': '34063029',
 'sentence': '(1) Background: A significantly reduced alanine aminotransferase (ALT) level is being recognized as a risk factor of increasing mortality in the elderly in relation to frailty.',
 'AB': '(1) Background: A significantly reduced alanine aminotransferase (ALT) level is being recognized as a risk factor of increasing mortality in the elderly in relation to frailty. In the elderly, both frailty and ischemic stroke are not only common, but are also associated with mortality. The aim of this research was to investigate whether a significantly reduced ALT level increases the all-cause mortality rate in the elderly with ischemic stroke. (2) Methods: Between February 2014 and April 2019, a retrospective study of 901 patients with ischemic stroke admitted to a university-affiliated hospital was conducted. Cox proportional hazard regression was used to determine whether a significantly reduced AL

In [326]:
pubtator_info_dict_TI={}
pubtator_info_dict_DP={}
for i in pubtator_info:
    pubtator_info_dict_TI[i.get('PMID')]=i.get('TI')
for i in pubtator_info:
    pubtator_info_dict_DP[i.get('PMID')]=i.get('DP')

In [327]:
Aging_Biomarkers_dict={}
for i in positive_relations:
    if not Aging_Biomarkers_dict.get('entity_one_transformed'):
        Aging_Biomarkers_dict[i.get('entity_one_transformed')]=[{
        'source entity':i.get('entity_one_transformed'),
            'relationship':i.get('triple')[1],
            'target entity':i.get('entity_two_transformed'),
            'sentence':i.get('sentence'),
            'source':i.get('entity_one'),
            'target':i.get('entity_two'),
            'source type':i.get('entity_one_type'),
            'target type':i.get('entity_two_type'),
            'PMID':i.get('PMID'),
            'DP':pubtator_info_dict_DP.get(i.get('PMID')),
            'date':i.get('date'),
            'TI':pubtator_info_dict_TI.get(i.get('PMID')),
            'TA':i.get('TA'),
            'IF':i.get('IF'),
            'IF5':i.get('IF5')
        }]

In [328]:
list(Aging_Biomarkers_dict.items())[0]

('GPT',
 [{'source entity': 'GPT',
   'relationship': 'predict',
   'target entity': 'Gallstones',
   'sentence': 'Elevated alanine aminotransferase (ALT > 120 U/L) was highly specific (97%) in predicting GP, with a positive likelihood ratio of 18.3.',
   'source': 'alanine aminotransferase',
   'target': 'GP',
   'source type': 'Gene',
   'target type': 'Disease',
   'PMID': '11418789',
   'DP': '2001 Jul',
   'date': 20010701,
   'TI': 'Gallstone pancreatitis: a community teaching hospital experience.',
   'TA': 'J Clin Gastroenterol',
   'IF': 3.174,
   'IF5': 3.226}])

In [331]:
len(Aging_Biomarkers_dict)

1603

In [330]:
Aging_Biomarkers_length=set()
for i,j in Aging_Biomarkers_dict.items():
    for k in j:
        Aging_Biomarkers_length.add(k.get('PMID'))
print(len(Aging_Biomarkers_length))

1309


In [332]:
json.dump(Aging_Biomarkers_dict,open('results/step5/Aging_Biomarkers.json','w'))

# Longevity_Biomarkers.json

In [333]:
Longevity_Biomarkers_dict={}
for i in negative_relations:
    if not Longevity_Biomarkers_dict.get('entity_one_transformed'):
        Longevity_Biomarkers_dict[i.get('entity_one_transformed')]=[{
        'source entity':i.get('entity_one_transformed'),
            'relationship':i.get('triple')[1],
            'target entity':i.get('entity_two_transformed'),
            'sentence':i.get('sentence'),
            'source':i.get('entity_one'),
            'target':i.get('entity_two'),
            'source type':i.get('entity_one_type'),
            'target type':i.get('entity_two_type'),
            'PMID':i.get('PMID'),
            'DP':pubtator_info_dict_DP.get(i.get('PMID')),
            'date':i.get('date'),
            'TI':pubtator_info_dict_TI.get(i.get('PMID')),
            'TA':i.get('TA'),
            'IF':i.get('IF'),
            'IF5':i.get('IF5')
        }]

In [334]:
list(Longevity_Biomarkers_dict.items())[0]

('Glucose',
 [{'source entity': 'Glucose',
   'relationship': 'prevent',
   'target entity': 'Diabetes Mellitus',
   'sentence': 'The results suggest that appropriate blood glucose control may be key to preventing intervertebral disc degeneration in diabetic patients.',
   'source': 'glucose',
   'target': 'diabetic',
   'source type': 'Carbohydrate',
   'target type': 'Disease',
   'PMID': '35005809',
   'DP': '2022 Oct',
   'date': 20221001,
   'TI': 'In vitro and in vivo effects of hyperglycemia and diabetes mellitus on nucleus pulposus cell senescence.',
   'TA': 'J Orthop Res',
   'IF': 3.102,
   'IF5': 3.579}])

In [335]:
len(Longevity_Biomarkers_dict)

452

In [336]:
Longevity_Biomarkers_length=set()
for i,j in Longevity_Biomarkers_dict.items():
    for k in j:
        Longevity_Biomarkers_length.add(k.get('PMID'))
print(len(Longevity_Biomarkers_length))

421


In [337]:
json.dump(Longevity_Biomarkers_dict,open('results/step5/Longevity_Biomarkers.json','w'))